In [229]:
import openclean
import glob
import pandas as pd
import pyspark 
import numpy as np

In [230]:
pyspark.__version__

'3.2.0'

In [231]:
np.__version__

'1.21.4'

In [232]:
pd.__version__

'1.3.4'

In [233]:
import glob

In [234]:
glob.glob("*")

['DOB_Job_Application_Filings.csv',
 'DOB_Job_Cleaning.ipynb',
 'ic3t-wcy2.tsv.gz',
 'README.md',
 'wordcount.py']

In [235]:
df  = pd.read_csv('./DOB_Job_Application_Filings.csv', dtype='object')

In [236]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1774873 entries, 0 to 1774872
Data columns (total 96 columns):
 #   Column                         Non-Null Count    Dtype 
---  ------                         --------------    ----- 
 0   Job #                          1774873 non-null  object
 1   Doc #                          1774873 non-null  object
 2   Borough                        1774873 non-null  object
 3   House #                        1774867 non-null  object
 4   Street Name                    1774867 non-null  object
 5   Block                          1774121 non-null  object
 6   Lot                            1774117 non-null  object
 7   Bin #                          1774873 non-null  object
 8   Job Type                       1774873 non-null  object
 9   Job Status                     1774873 non-null  object
 10  Job Status Descrp              1774873 non-null  object
 11  Latest Action Date             1774873 non-null  object
 12  Building Type               

In [237]:
df.count()

Job #                   1774873
Doc #                   1774873
Borough                 1774873
House #                 1774867
Street Name             1774867
                         ...   
GIS_LONGITUDE           1769216
GIS_COUNCIL_DISTRICT    1769216
GIS_CENSUS_TRACT        1769216
GIS_NTA_NAME            1769216
GIS_BIN                 1752744
Length: 96, dtype: int64

### if any rows are complete duplicates, drop them

In [238]:
df = df.drop_duplicates()

In [239]:
df

,Job #,Doc #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Job Status,...,SPECIAL_ACTION_STATUS,SPECIAL_ACTION_DATE,BUILDING_CLASS,JOB_NO_GOOD_COUNT,GIS_LATITUDE,GIS_LONGITUDE,GIS_COUNCIL_DISTRICT,GIS_CENSUS_TRACT,GIS_NTA_NAME,GIS_BIN
0,440673512,01,QUEENS,10040,222 STREET,10780,00019,4231025,A2,R,...,N,NaN,A1,0,40.714365,-73.731171,27,568,Queens Village,4231025
1,421133972,02,QUEENS,88-36,139TH STREET,09620,00033,4623670,NB,P,...,N,NaN,G7,0,40.703428,-73.813796,24,214,Briarwood-Jamaica Hills,NaN
2,421133972,03,QUEENS,88-36,139TH STREET,09620,00033,4623670,NB,P,...,N,NaN,G7,0,40.703428,-73.813796,24,214,Briarwood-Jamaica Hills,NaN
3,210182309,02,BRONX,97,WEST 169 STREET,02519,00027,2130622,NB,P,...,N,NaN,V0,0,40.839538,-73.923177,16,211,Highbridge,NaN
4,210182309,03,BRONX,97,WEST 169 STREET,02519,00027,2130622,NB,P,...,N,NaN,V0,0,40.839538,-73.923177,16,211,Highbridge,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1774868,302042858,01,BROOKLYN,2770,OCEAN AVENUE,07404,00020,3202124,A3,X,...,N,NaN,D1,0,40.593831,-73.950368,48,592,Sheepshead Bay-Gerritsen Beach-Manhattan Beach,3202124
1774869,302042867,01,BROOKLYN,313,76 STREET,05940,00078,3148750,A2,R,...,N,NaN,B1,0,40.630611,-74.027435,43,66,Bay Ridge,3148750
1774870,302042876,01,BROOKLYN,1658,66 STREET,05559,00033,3134197,A2,X,...,N,NaN,C3,0,40.620486,-73.994067,38,252,Bensonhurst West,3134197
1774871,302042885,01,BROOKLYN,173,LEXINGTON AVENUE,01968,00043,3056361,A2,Q,...,N,NaN,F9,0,40.687159,-73.955516,36,233,Bedford,3056361


In [240]:
df.columns#[5,6,13,14,20,21,32,38,59,60,68,74,75,76,77,79]

Index(['Job #', 'Doc #', 'Borough', 'House #', 'Street Name', 'Block', 'Lot',
       'Bin #', 'Job Type', 'Job Status', 'Job Status Descrp',
       'Latest Action Date', 'Building Type', 'Community - Board', 'Cluster',
       'Landmarked', 'Adult Estab', 'Loft Board', 'City Owned', 'Little e',
       'PC Filed', 'eFiling Filed', 'Plumbing', 'Mechanical', 'Boiler',
       'Fuel Burning', 'Fuel Storage', 'Standpipe', 'Sprinkler', 'Fire Alarm',
       'Equipment', 'Fire Suppression', 'Curb Cut', 'Other',
       'Other Description', 'Applicant's First Name', 'Applicant's Last Name',
       'Applicant Professional Title', 'Applicant License #',
       'Professional Cert', 'Pre- Filing Date', 'Paid', 'Fully Paid',
       'Assigned', 'Approved', 'Fully Permitted', 'Initial Cost',
       'Total Est. Fee', 'Fee Status', 'Existing Zoning Sqft',
       'Proposed Zoning Sqft', 'Horizontal Enlrgmt', 'Vertical Enlrgmt',
       'Enlargement SQ Footage', 'Street Frontage', 'ExistingNo. of Stories',
  

In [241]:
df.head()

,Job #,Doc #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Job Status,...,SPECIAL_ACTION_STATUS,SPECIAL_ACTION_DATE,BUILDING_CLASS,JOB_NO_GOOD_COUNT,GIS_LATITUDE,GIS_LONGITUDE,GIS_COUNCIL_DISTRICT,GIS_CENSUS_TRACT,GIS_NTA_NAME,GIS_BIN
0,440673512,01,QUEENS,10040,222 STREET,10780,00019,4231025,A2,R,...,N,NaN,A1,0,40.714365,-73.731171,27,568,Queens Village,4231025
1,421133972,02,QUEENS,88-36,139TH STREET,09620,00033,4623670,NB,P,...,N,NaN,G7,0,40.703428,-73.813796,24,214,Briarwood-Jamaica Hills,NaN
2,421133972,03,QUEENS,88-36,139TH STREET,09620,00033,4623670,NB,P,...,N,NaN,G7,0,40.703428,-73.813796,24,214,Briarwood-Jamaica Hills,NaN
3,210182309,02,BRONX,97,WEST 169 STREET,02519,00027,2130622,NB,P,...,N,NaN,V0,0,40.839538,-73.923177,16,211,Highbridge,NaN
4,210182309,03,BRONX,97,WEST 169 STREET,02519,00027,2130622,NB,P,...,N,NaN,V0,0,40.839538,-73.923177,16,211,Highbridge,NaN


## Take an initial look at Job #

In [242]:
df['Job #'].value_counts(dropna=False)

103408705    20
122705000    13
120438240    13
320843110    12
302136203    11
             ..
220173489     1
220173498     1
420542960     1
121002379     1
302042894     1
Name: Job #, Length: 1584321, dtype: int64

In [243]:
df['Job #'].min()

'100321046'

In [244]:
df['Job #'].max()

'577777776'

In [245]:
df.loc[df['Job #'].str.startswith('0')]

,Job #,Doc #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Job Status,...,SPECIAL_ACTION_STATUS,SPECIAL_ACTION_DATE,BUILDING_CLASS,JOB_NO_GOOD_COUNT,GIS_LATITUDE,GIS_LONGITUDE,GIS_COUNCIL_DISTRICT,GIS_CENSUS_TRACT,GIS_NTA_NAME,GIS_BIN


In [246]:
df['Job #'].count()

1774873

In [247]:
df['Job #'].nunique()

1584321

In [248]:
df.loc[df['Job #']=='103408705']

,Job #,Doc #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Job Status,...,SPECIAL_ACTION_STATUS,SPECIAL_ACTION_DATE,BUILDING_CLASS,JOB_NO_GOOD_COUNT,GIS_LATITUDE,GIS_LONGITUDE,GIS_COUNCIL_DISTRICT,GIS_CENSUS_TRACT,GIS_NTA_NAME,GIS_BIN
44593,103408705,02,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,R,...,A,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
44594,103408705,03,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,R,...,A,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
77020,103408705,08,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,P,...,NaN,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
77024,103408705,09,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,P,...,NaN,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
77103,103408705,04,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,P,...,NaN,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
77107,103408705,05,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,P,...,NaN,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
77109,103408705,06,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,P,...,NaN,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
77111,103408705,12,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,P,...,NaN,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
77115,103408705,13,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,P,...,NaN,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
77120,103408705,14,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,P,...,NaN,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345


In [249]:
df.describe()

,Job #,Doc #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Job Status,...,SPECIAL_ACTION_STATUS,SPECIAL_ACTION_DATE,BUILDING_CLASS,JOB_NO_GOOD_COUNT,GIS_LATITUDE,GIS_LONGITUDE,GIS_COUNCIL_DISTRICT,GIS_CENSUS_TRACT,GIS_NTA_NAME,GIS_BIN
count,1774873,1774873,1774873,1774867,1774867,1774121,1774117,1774873,1774873,1774873,...,1754670,172200,1768455,1774873,1769216,1769216,1769216,1769216,1769216,1752744
unique,1584321,20,5,35646,29553,13816,1971,363786,9,17,...,4,6374,199,13,217561,228338,51,1328,194,342803
top,103408705,01,MANHATTAN,1,BROADWAY,00016,00001,1015862,A2,X,...,N,10/24/2012,R0,0,40.748276,-73.984690,4,7,Midtown-Midtown South,3000000
freq,20,1583895,775342,17359,52273,6808,216451,2413,1117095,1050470,...,1519423,5513,211273,1772958,2409,2413,236661,16439,153871,15627


In [250]:
df.columns[:20]

Index(['Job #', 'Doc #', 'Borough', 'House #', 'Street Name', 'Block', 'Lot',
       'Bin #', 'Job Type', 'Job Status', 'Job Status Descrp',
       'Latest Action Date', 'Building Type', 'Community - Board', 'Cluster',
       'Landmarked', 'Adult Estab', 'Loft Board', 'City Owned', 'Little e'],
      dtype='object')

## Describe columns in groups so they fit on screen

In [251]:
df[df.columns[:20]].describe()

,Job #,Doc #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Job Status,Job Status Descrp,Latest Action Date,Building Type,Community - Board,Cluster,Landmarked,Adult Estab,Loft Board,City Owned,Little e
count,1774873,1774873,1774873,1774867,1774867,1774121,1774117,1774873,1774873,1774873,1774873,1774873,1774873,1773578,1131380,1670754,1581426,1400834,149402,1046451
unique,1584321,20,5,35646,29553,13816,1971,363786,9,17,17,7374,2,106,2,4,2,2,4,5
top,103408705,01,MANHATTAN,1,BROADWAY,00016,00001,1015862,A2,X,SIGNED OFF,10/13/2017,OTHERS,105,N,N,N,N,Y,N
freq,20,1583895,775342,17359,52273,6808,216451,2413,1117095,1050470,1050470,793,1389737,210723,1122909,1461499,1581253,1393398,149399,989667


In [252]:
# Notes:
# Building Type looks binary and has 2 values + maybe NAN
# Cluster looks binary and has 2 values + maybe NAN
# Landmarked looks binary and has 4 values + maybe NAN
# Adult Estab looks binary and has 2 values + maybe NAN
# Loft Board looks binary and has 2 values + maybe NAN
# City Owned looks binary and has 4 values + maybe NAN
# Little e looks binary and has 5 values + maybe NAN


In [253]:
df[df.columns[20:40]].describe()

,PC Filed,eFiling Filed,Plumbing,Mechanical,Boiler,Fuel Burning,Fuel Storage,Standpipe,Sprinkler,Fire Alarm,Equipment,Fire Suppression,Curb Cut,Other,Other Description,Applicant's First Name,Applicant's Last Name,Applicant Professional Title,Applicant License #,Professional Cert
count,602206,1081520,555711,284299,49959,21451,14255,14386,116387,87703,306812,43370,61213,1041766,1039964,1774814,1774861,1774790,1725704,1396678
unique,2,2,1,1,1,1,2,1,1,1,1,1,1,1,16626,17354,35220,913,27089,5
top,Y,Y,X,X,X,X,X,X,X,X,X,X,X,X,GEN. CONSTR.,MICHAEL,HOQUE,PE,083278,Y
freq,384720,1081518,555711,284299,49959,21451,14254,14386,116387,87703,306812,43370,61213,1041766,316607,51059,19159,924260,19195,859594


In [254]:
# PC Filed -- Other all look binary, and have 1-2 values + maybe NAN
#
# Take a look at Other Description for weird strings

#  Lots of the same First and Last name

# Check names and titles

# APPLICATNT LICENCSE # NEEDS TO BE A STRING TO PERSERVE THE 0 ON IT (PROBABLY)

# Professional Cert looks binary and has 5 values + maybe NAN



In [255]:
df[df.columns[40:60]].describe()

,Pre- Filing Date,Paid,Fully Paid,Assigned,Approved,Fully Permitted,Initial Cost,Total Est. Fee,Fee Status,Existing Zoning Sqft,Proposed Zoning Sqft,Horizontal Enlrgmt,Vertical Enlrgmt,Enlargement SQ Footage,Street Frontage,ExistingNo. of Stories,Proposed No. of Stories,Existing Height,Proposed Height,Existing Dwelling Units
count,1774873,1765805,1768295,1195882,1444999,1309748,1774873,1774873,1774873,1774873,1774873,45822,32149,1774873,1774873,1774873,1774873,1774873,1774873,644113
unique,6623,6642,6709,6782,6888,6122,98627,93203,3,18057,24620,2,2,6970,1184,152,319,945,1102,1082
top,06/28/2019,06/28/2019,06/28/2019,07/01/2009,06/26/2008,11/12/2015,$0.00,$100.00,STANDARD,0,0,Y,Y,0,0,0,0,0,0,1
freq,1717,1700,1702,575,663,487,477561,144802,1600970,1704887,1673008,45815,32140,1713529,1677540,821455,396318,821473,454072,117787


In [256]:
# Need to convert date columns to pd.datetime
# RENAME PAID TO PAID DATE
# RENAME FULLY PAID TO FULLY PAID DATE
# RENAME ASSISGNED TO ASSIGNED DATE
# RENAME APPROVED TO APPROVED DATE

# CHECK COHERENCE OF PAID DATE <= FULLY PAID DATE
# CHECK COHERENCE OF PRE FILING DATE <= PAID DATE
# CHECK COHERENCE OF ASSIGNED DATE <= APPROVED DATE
# 

# REMOVE $ FROM Initial Cost and Total Est. Fee, and put them in column name, convert values to floats

# Check What fee status is

# Check Existing Zoning Sqft, Propsed Zoning Sqft, Enlargement SQ Footage for reasonable values
# Change either Sqft to SQ Footage or vvice-versa

# Horizontal Enlrgmt and Vertical Enlrgmt are booleans + NAN
# Change Enlrgmt to Enlargement

#Chcek ExisitngNo. of Stories and proposed No. of Stories for reasonableness
# Add space between Existing and No.
# Change either Job# to Job No. or vice versa
# maybe just change all the No./# to "number"

# Check Existing and proposed height for reasonableness. Add unit to column name

# check Existing Dwelling Units for reasonableness 



In [257]:
df[df.columns[60:80]].describe()

,Proposed Dwelling Units,Existing Occupancy,Proposed Occupancy,Site Fill,Zoning Dist1,Zoning Dist2,Zoning Dist3,Special District 1,Special District 2,Owner Type,Non-Profit,Owner's First Name,Owner's Last Name,Owner's Business Name,Owner's House Number,Owner'sHouse Street Name,City,State,Zip,Owner'sPhone #
count,918750,1270615,1062769,1425022,1440085,193634,10263,262140,79918,1701479,1703326,1774593,1774658,1424258,1003,1003,1003,1003,1001,1763406
unique,1173,47,49,5,229,196,135,116,15,14,2,93118,156871,412722,549,756,114,9,212,359280
top,1,RES,RES,NOT APPLICABLE,R6,R8B,C6-6,MID,POPS,CORPORATION,N,MICHAEL,SINGH,OWNER,2,555 WEST 23RD STREET,BROOKLYN,NY,11201,7184728000
freq,172062,411653,289997,717129,149653,19305,1554,84262,27848,575535,1594188,40957,23629,11056,19,17,309,981,46,12732


In [258]:
# check Proposed Dwelling Units
## Why does Existing Occupancy have fewer cats than Proposed Occupancy. Check those.

# What is Site Fill. 

# Get list of NYC ZOning Districts and Special Districts

# Checmk Owner Typer for spelling issues

# Non Profit is binary

# Check Owners's First and last name

# Owner'sBuisness Name should not be "OWNER"

# Owner's house number, streeet name, city, state and zip have almost no values

# Why is the same phone number so common

In [259]:
df[df.columns[80:100]].describe()

,Job Description,DOBRunDate,JOB_S1_NO,TOTAL_CONSTRUCTION_FLOOR_AREA,WITHDRAWAL_FLAG,SIGNOFF_DATE,SPECIAL_ACTION_STATUS,SPECIAL_ACTION_DATE,BUILDING_CLASS,JOB_NO_GOOD_COUNT,GIS_LATITUDE,GIS_LONGITUDE,GIS_COUNCIL_DISTRICT,GIS_CENSUS_TRACT,GIS_NTA_NAME,GIS_BIN
count,1594531,1774873,1774873,1774873,1774873,1063365,1754670,172200,1768455,1774873,1769216,1769216,1769216,1769216,1769216,1752744
unique,1247253,298,1774873,26598,13,5920,4,6374,199,13,217561,228338,51,1328,194,342803
top,PROPOSED INSTALLATION OF HEAVY DUTY SIDEWALK S...,04/28/2021 00:00:00,3294002,0,0,02/12/2018,N,10/24/2012,R0,0,40.748276,-73.984690,4,7,Midtown-Midtown South,3000000
freq,3985,1674972,1,1619567,1752667,603,1519423,5513,211273,1772958,2409,2413,236661,16439,153871,15627


In [260]:

# That is a lot of unique job descriptions

# Add spaces to DOBRunDate name
# make DOBRunDate a datetime

# What is Job_S1_NO. It uses underscores.

# All the remaining columns have ALL CAPS NAMES WITH UNDERSCORES 
# TOTAL_CONSTRUCTION_FLOOR_AREA, WITHDRAWAL_FLAG

# SIGNOFF_DATE needs to be datetime
# SPECIAL_ACTION_STATUS
# SPECIAL_ACTION_DATE needs to be datetime
# BUILDING_CLASS
# What is JOB_NO_GOOD_COUNT
#
# maybe need GIS DATA
# GIS_LATITUDE
# GIS_LONGITUDE
# GIS_COUNCIL_DISTRICT
# GIS_CENSUS_TRACT
# GIS_NTA_NAME
# GIS_BIN
# 

In [261]:
df.columns

Index(['Job #', 'Doc #', 'Borough', 'House #', 'Street Name', 'Block', 'Lot',
       'Bin #', 'Job Type', 'Job Status', 'Job Status Descrp',
       'Latest Action Date', 'Building Type', 'Community - Board', 'Cluster',
       'Landmarked', 'Adult Estab', 'Loft Board', 'City Owned', 'Little e',
       'PC Filed', 'eFiling Filed', 'Plumbing', 'Mechanical', 'Boiler',
       'Fuel Burning', 'Fuel Storage', 'Standpipe', 'Sprinkler', 'Fire Alarm',
       'Equipment', 'Fire Suppression', 'Curb Cut', 'Other',
       'Other Description', 'Applicant's First Name', 'Applicant's Last Name',
       'Applicant Professional Title', 'Applicant License #',
       'Professional Cert', 'Pre- Filing Date', 'Paid', 'Fully Paid',
       'Assigned', 'Approved', 'Fully Permitted', 'Initial Cost',
       'Total Est. Fee', 'Fee Status', 'Existing Zoning Sqft',
       'Proposed Zoning Sqft', 'Horizontal Enlrgmt', 'Vertical Enlrgmt',
       'Enlargement SQ Footage', 'Street Frontage', 'ExistingNo. of Stories',
  

In [262]:
def show_vals(column_name, show_rows=10, df=df):
    print("Top {} {}:\n".format(show_rows, column_name))
    print(df[column_name].value_counts(dropna=False)[:show_rows])
    print()

### House numbers can't be strings because some have dashes

In [263]:
show_vals('House #')

Top 10 House #:

1      17359
200    11853
100     8114
11      7231
10      7191
55      7166
40      7105
30      7098
2       6428
150     6380
Name: House #, dtype: int64



### House numbers shouldn't be words

In [264]:
df.loc[(~df['House #'].isna())
       &(df['House #'].str.isalpha())]['House #']

15453            ONE
1174319         PIER
1181056         PIER
1345828          ONE
1345994          ONE
1378376          ONE
1400308          ONE
1405901          ONE
1513818    MANHATTAN
Name: House #, dtype: object

In [265]:
df['House #'] = df['House #'].astype('str')

In [266]:
df.loc[(~df['House #'].isna())
       &(~df['House #'].str.isdigit())]['House #']

1            88-36
2            88-36
5           215-02
6            34-74
15           34-74
            ...   
1774660    186-200
1774661    186-200
1774687    372-380
1774710    633-639
1774855      26-28
Name: House #, Length: 332189, dtype: object

### These look like Apartment numbers mostly

In [267]:
df.loc[(~df['House #'].isna())
       &(~df['House #'].str.isdigit())
      &(~df['House #'].str.contains('-'))]['House #']

37           215B
71         270GAR
76         9 WEST
263         1715A
264         1715A
            ...  
1773945        7A
1774110      486A
1774528     539 A
1774534      140B
1774612      277A
Name: House #, Length: 11052, dtype: object

## Looking at Binary columns:

In [268]:
show_vals('Landmarked')
show_vals('Adult Estab')
show_vals('Loft Board')
show_vals('City Owned')
show_vals('Little e')
show_vals('PC Filed')
show_vals('eFiling Filed')


Top 10 Landmarked:

N      1461499
Y       208863
NaN     104119
L          369
C           23
Name: Landmarked, dtype: int64

Top 10 Adult Estab:

N      1581253
NaN     193447
Y          173
Name: Adult Estab, dtype: int64

Top 10 Loft Board:

N      1393398
NaN     374039
Y         7436
Name: Loft Board, dtype: int64

Top 10 City Owned:

NaN    1625471
Y       149399
I            1
M            1
N            1
Name: City Owned, dtype: int64

Top 10 Little e:

N      989667
NaN    728422
Y       36056
H       20723
M           4
X           1
Name: Little e, dtype: int64

Top 10 PC Filed:

NaN    1172667
Y       384720
N       217486
Name: PC Filed, dtype: int64

Top 10 eFiling Filed:

Y      1081518
NaN     693353
R            2
Name: eFiling Filed, dtype: int64



In [269]:
show_vals('Plumbing')
show_vals('Mechanical')
show_vals('Boiler')
show_vals('Fuel Burning')
show_vals('Fuel Storage')
show_vals('Standpipe')
show_vals('Sprinkler')
show_vals('Fire Alarm')
show_vals('Equipment')
show_vals('Fire Suppression')
show_vals('Curb Cut')
show_vals('Other')

Top 10 Plumbing:

NaN    1219162
X       555711
Name: Plumbing, dtype: int64

Top 10 Mechanical:

NaN    1490574
X       284299
Name: Mechanical, dtype: int64

Top 10 Boiler:

NaN    1724914
X        49959
Name: Boiler, dtype: int64

Top 10 Fuel Burning:

NaN    1753422
X        21451
Name: Fuel Burning, dtype: int64

Top 10 Fuel Storage:

NaN    1760618
X        14254
N            1
Name: Fuel Storage, dtype: int64

Top 10 Standpipe:

NaN    1760487
X        14386
Name: Standpipe, dtype: int64

Top 10 Sprinkler:

NaN    1658486
X       116387
Name: Sprinkler, dtype: int64

Top 10 Fire Alarm:

NaN    1687170
X        87703
Name: Fire Alarm, dtype: int64

Top 10 Equipment:

NaN    1468061
X       306812
Name: Equipment, dtype: int64

Top 10 Fire Suppression:

NaN    1731503
X        43370
Name: Fire Suppression, dtype: int64

Top 10 Curb Cut:

NaN    1713660
X        61213
Name: Curb Cut, dtype: int64

Top 10 Other:

X      1041766
NaN     733107
Name: Other, dtype: int64



In [270]:
# Many of these values are the same but spelled differently
show_vals('Other Description')

Top 10 Other Description:

NaN                734909
GEN. CONSTR.       316607
GC                 116243
GEN. CONSTR         87350
STRUCTURAL          55577
BPP                 50690
GEN CONST           42153
SOLAR TAX           33572
GEN.CONSTRUCTN.     26544
FACADE              22633
Name: Other Description, dtype: int64



In [271]:
# Should remove "MR" when found
show_vals("Applicant's First Name")

Top 10 Applicant's First Name:

MICHAEL    51059
JOHN       44296
ROBERT     41756
DAVID      35926
PAUL       32698
ANTHONY    29881
JAMES      27063
JOSEPH     26992
THOMAS     24209
PETER      21917
Name: Applicant's First Name, dtype: int64



In [272]:
df["Applicant's First Name"] = df["Applicant's First Name"].astype('str')

In [273]:
# Unclear if these are actual first names mostly (Except for WU (woody))
df.loc[df["Applicant's First Name"].str.contains(" ")]["Applicant's First Name"].value_counts()

MD TOHFAZ      3090
KO SHEK        1633
JUNG WOR       1228
PING MAN       1133
WU (WOODY)     1119
               ... 
JOHN OLIVER       1
JAMES DEAN        1
CHOON HONG        1
LLOYD MACK        1
JOAN BOOYE        1
Name: Applicant's First Name, Length: 2790, dtype: int64

In [274]:
df["Applicant's Last Name"] = df["Applicant's Last Name"].astype('str')

In [275]:
# At least one of thesse is not all caps, and clearly there are periods in some
show_vals("Applicant's Last Name")

Top 10 Applicant's Last Name:

HOQUE         19159
KATZ          15126
LEE           14513
RUDIKOFF      12109
GERAZOUNIS    11126
BAILEY        10826
CALIENDO       9306
CHEN           9110
SYED-NAQVI     7299
MASS           6665
Name: Applicant's Last Name, dtype: int64



In [276]:
# Most of these are the same
show_vals("Applicant Professional Title", show_rows=50)

Top 50 Applicant Professional Title:

PE                 924260
RA                 796695
GC                  12762
OT                  10619
G.C                  5707
CONTRACTOR           3314
DEMO CONTRACTOR      2087
DEMO                 1973
PERMIT BROKER        1765
GEN CONTRACTOR       1504
G.C.                 1492
Contractor           1355
OWNER                1228
Demo Contractor       639
X                     534
DEMO CONTR            476
GEN. CONTRACTOR       451
SIGN HANGER           350
GENERAL CONTRAC       348
LA                    267
SIGN PERMIT AGT       248
Demolition Cont       242
SUBCONTRACTOR         232
Owner                 194
G. CONTRACTOR         177
GEN.CONTRACTOR        172
PLLC                  161
LAND SURVEYOR         131
G. C.                 129
General Contrac       124
GEN CONT              108
DEMO CONT             106
DEMO. CONTR           105
contractor            100
Gen. Contractor        95
G C                    94
G CONTRACTOR           93


In [277]:
show_vals('Applicant License #', show_rows=20)

Top 20 Applicant License #:

NaN       49169
083278    19195
050579    16342
066709    12092
013529    10886
082770    10711
020419    10230
020241     9350
080123     8246
038964     7522
045101     6732
058375     6614
076082     6570
066791     6334
035290     6109
053541     6031
054953     5945
029017     5627
075316     5354
071719     5290
Name: Applicant License #, dtype: int64



In [278]:
# Some of these are obviously wrong
show_vals('Professional Cert')

Top 10 Professional Cert:

Y      859594
N      537080
NaN    378195
J           2
F           1
R           1
Name: Professional Cert, dtype: int64



## Checking money consistency

In [279]:
show_vals("Initial Cost")

Top 10 Initial Cost:

$0.00        477561
$10000.00     57794
$1000.00      55261
$3000.00      51157
$5000.00      50529
$20000.00     43246
$15000.00     38850
$25000.00     35438
$30000.00     30253
$50000.00     30054
Name: Initial Cost, dtype: int64



In [280]:
df['Initial Cost'] = df['Initial Cost'].astype('str')

In [281]:
df.loc[~df["Initial Cost"].str.contains(".")]["Initial Cost"].value_counts()

Series([], Name: Initial Cost, dtype: int64)

In [282]:
show_vals("Total Est. Fee")
show_vals("Fee Status")

Top 10 Total Est. Fee:

$100.00    144802
$0.00       58846
$160.00     58050
$130.00     55733
$225.00     42217
$140.00     33609
$200.00     31413
$191.50     26906
$294.50     20432
$346.00     18524
Name: Total Est. Fee, dtype: int64

Top 10 Fee Status:

STANDARD    1600970
EXEMPT       168158
DEFERRED       5745
Name: Fee Status, dtype: int64



In [283]:
df['Total Est. Fee'] = df['Total Est. Fee'].astype('str')

In [284]:
df.loc[~df["Total Est. Fee"].str.contains(".")]["Total Est. Fee"].value_counts()

Series([], Name: Total Est. Fee, dtype: int64)

The money formatting is pretty consistent. Nothing looks out of place

## Checking owner's information

In [285]:
show_vals("Owner's Business Name")
show_vals("Owner's House Number")
show_vals("Owner'sHouse Street Name")
show_vals("City ")
show_vals("State")
show_vals("Zip")

Top 10 Owner's Business Name:

NaN                                 350615
OWNER                                11056
NY SCHOOL CONSTRUCTION AUTHORITY      9758
NONE                                  9558
NYC SCA                               9228
VORNADO OFFICE MANAGEMENT             6990
NYCHA                                 6652
SL GREEN REALTY CORP.                 5696
NYC HOUSING AUTHORITY                 5696
NEW YORK CITY HOUSING AUTHORITY       4737
Name: Owner's Business Name, dtype: int64

Top 10 Owner's House Number:

NaN      1773870
2             19
30-30         18
555           17
22            16
1950          16
100           16
10            12
250           11
32            11
Name: Owner's House Number, dtype: int64

Top 10 Owner'sHouse Street Name:

NaN                            1773870
555 WEST 23RD STREET                17
22 CHAPEL STREET                    10
1559 49TH STREET                     9
30-30 THOMSON AVENUE                 9
35 N TYSON AVENUE STE#102

In [286]:
#TO DO:
"""
Owner's Business Name: 
    Fix duplicate names + awkward names like OWNER

Owner's House Number: 
    Same issue as House #

Owner'sHouse Street Name:
    Change name to add the space between Owner's and House

City :
    look at lecture slide #45+ in lec 8-data-profiling-cleaning
    levenshtein?
    fix name because it has a trailing space in City 

State:
    Get a list of 50 state codes and remove all the ones that are not in there

Zip:
    A bit tricky because some houses are outside NYC so you can't just get a NYC zip code. Would have to get
    zip code of every state...
"""

"\nOwner's Business Name: \n    Fix duplicate names + awkward names like OWNER\n\nOwner's House Number: \n    Same issue as House #\n\nOwner'sHouse Street Name:\n    Change name to add the space between Owner's and House\n\nCity :\n    look at lecture slide #45+ in lec 8-data-profiling-cleaning\n    levenshtein?\n    fix name because it has a trailing space in City \n\nState:\n    Get a list of 50 state codes and remove all the ones that are not in there\n\nZip:\n    A bit tricky because some houses are outside NYC so you can't just get a NYC zip code. Would have to get\n    zip code of every state...\n"

## Looking at Phone Numbers:

In [287]:
show_vals("Owner'sPhone #")

Top 10 Owner'sPhone #:

7184728000    12732
NaN           11467
2128947000    10345
2128637625     5610
7183924921     4411
2122161664     4190
2124072400     3922
7184728534     3346
2125295688     3313
5169355555     3288
Name: Owner'sPhone #, dtype: int64



In [288]:
df["Owner'sPhone #"] = df["Owner'sPhone #"].astype('str')

In [289]:
df.loc[df["Owner'sPhone #"].str.contains("7184728000")][["Owner's First Name", "Owner's Last Name","Owner's Business Name", "Owner'sPhone #"]]

,Owner's First Name,Owner's Last Name,Owner's Business Name,Owner'sPhone #
6,JOSEPH,SCALISI,SCHOOL CONSTRUCTION AUTHORITY,7184728000
15,JOSEPH,SCALISI,SCHOOL CONSTRUCTION AUTHORITY,7184728000
34,JOSEPH,SCALISI,NYCSCA,7184728000
74,SRIDHAR,DULPETA-RAMA,SCHOOL CONSTRUCTION AUTHORITY,7184728000
129,EFFIE,TSITIRIDIS,SCHOOL CONSTRUCTION AUTHORITY,7184728000
...,...,...,...,...
1770710,MICHAEL,KALEDA,NYCSA,7184728000
1770851,AMY,LINDEN,NYS BOARD OF EDUCATION,7184728000
1770852,AMY,LINDEN,NYS BOARD OF EDUCATION,7184728000
1771916,AMY,LINDEN,NYS BOARD OF EDUCATION,7184728000


In [290]:
df.loc[df["Owner'sPhone #"].isna()]

,Job #,Doc #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Job Status,...,SPECIAL_ACTION_STATUS,SPECIAL_ACTION_DATE,BUILDING_CLASS,JOB_NO_GOOD_COUNT,GIS_LATITUDE,GIS_LONGITUDE,GIS_COUNCIL_DISTRICT,GIS_CENSUS_TRACT,GIS_NTA_NAME,GIS_BIN


In [291]:
df.loc[~df["Owner'sPhone #"].isna() & df["Owner'sPhone #"].str.contains("-")]["Owner'sPhone #"].value_counts()

212582-455    3
212490192-    2
71852751-1    2
0-            2
21255736--    2
212816-22-    1
21267955--    1
212-020844    1
631-323395    1
212808090-    1
2123-51347    1
212269689-    1
71835636-7    1
718888-600    1
71888422--    1
71839130-3    1
718291550-    1
212675200-    1
215-29        1
718767168-    1
149-62        1
157-15        1
146-02        1
163-07        1
243-17        1
718-942300    1
175-27        1
191-15        1
150-24        1
71820585-1    1
718343645-    1
212863749-    1
102-18        1
71845657-6    1
242-01        1
Name: Owner'sPhone #, dtype: int64

In [292]:
df.loc[~df["Owner'sPhone #"].isna() & df["Owner'sPhone #"].str.contains(" ")]["Owner'sPhone #"].value_counts()

212  58664    7
212483 077    6
212 929318    5
212 996510    4
718391 105    4
             ..
917939  99    1
212    279    1
718  35377    1
718 555555    1
9176 96 89    1
Name: Owner'sPhone #, Length: 115, dtype: int64

Goals:

Remove all "-" and " "

Check if the 1st value is a 0 or 1, if so, remove it

Remove all with less than or greater than 10 digits


## Cleaning phone number

In [293]:
df["Owner'sPhone #"] = df["Owner'sPhone #"].str.replace("-", '')
df.loc[~df["Owner'sPhone #"].isna() & df["Owner'sPhone #"].str.contains("-")]["Owner'sPhone #"].value_counts()

Series([], Name: Owner'sPhone #, dtype: int64)

In [294]:
df["Owner'sPhone #"] = df["Owner'sPhone #"].str.replace(" ", '')
df.loc[~df["Owner'sPhone #"].isna() & df["Owner'sPhone #"].str.contains(" ")]["Owner'sPhone #"].value_counts()

Series([], Name: Owner'sPhone #, dtype: int64)

In [344]:
df["Owner'sPhone #"] = df["Owner'sPhone #"].astype('str')
df.loc[~df["Owner'sPhone #"].isna() & ((df["Owner'sPhone #"].str[0] == "0") | (df["Owner'sPhone #"].str[0] == "1") | (df["Owner'sPhone #"].apply(len) != 10)), ["Owner'sPhone #"]] = np.nan

In [348]:
df["Owner'sPhone #"] = df["Owner'sPhone #"].astype('str')
df.loc[(df["Owner'sPhone #"].apply(len) != 10)]["Owner'sPhone #"]

226        nan
637        nan
655        nan
683        nan
2026       nan
          ... 
1774748    nan
1774757    nan
1774782    nan
1774845    nan
1774870    nan
Name: Owner'sPhone #, Length: 15423, dtype: object

## Renaming columns

In [350]:
df = df.rename(columns={"Owner'sPhone #": "Owner's Phone #"})